These lines code blocks read in all of the weather files from Sheffield. I used FTP (Filezilla) to do a bulk doanload from the CEDA Archive website. This is essentially the same script we ended up with last week

In [1]:
import glob
import pandas as pd
file_list = glob.glob(r"/content/drive/MyDrive/2022-2023 Teaching/2_TRP479_Spatial_Data_Science/L7/qc-version-1/midas-open_uk-daily-temperature-obs_dv-202207_south-yorkshire_00525_sheffield_qcv-1*csv")


In [2]:
#creates an empty pandas dataframe to store our yearly data
summary_data = pd.DataFrame([])

# Repeat the analysis steps for each input file:
for fp in file_list:
   daily_mean=pd.DataFrame([])
# Read in only selected columns
   data = pd.read_csv(fp,
                    usecols=['ob_end_time', 'id_type', 'ob_hour_count',
       'src_id',  'max_air_temp',
       'min_air_temp'], skiprows=90)

# 'drop' the last row of data in our data frame becasue we know it contains the 'end data' flag
   data.drop(len(data)-1,axis = 'index',inplace =True)

# Rename the columns
   new_names = {"ob_end_time" : "TIMESTAMP", "max_air_temp" :"MAX", "min_air_temp": "MIN"}
   data = data.rename(columns=new_names)






# Slice the 'TIMESTAMP' string into DATE, YEAR, MONTH, DAY and TIME columns
   data["DATE"] = data["TIMESTAMP"].str.slice(start=0, stop=10)
   data["YEAR"] =data["TIMESTAMP"].str.slice(start=0, stop=4)
   data["MONTH"] = data["TIMESTAMP"].str.slice(start=5, stop=7)
   data["DAY"] = data["TIMESTAMP"].str.slice(start=8, stop=10)
   data["TIME"] = data["TIMESTAMP"].str.slice(start=11)


# Create empty column
   col_name = 'MID'
   data[col_name] = None

# Calculate 'average' temperature as midpoint between max and min readings
   data['MID'] = (data['MAX'] + data['MIN'])/2

   #group by month and day to get min and max values for 12 hour periods (needed for years with day and night records)
   grouped_month_day= data.groupby(by=['MONTH','DAY'])
   daily_mean['MIN']=grouped_month_day['MIN'].min() # gets MIN value for whole 24 hr period
   daily_mean['MAX']=grouped_month_day['MAX'].max() # gets MAX value for whole 24 hr period

   daily_mean['MID']=grouped_month_day['MID'].mean() #mean of 'day' and 'night' MID values
   daily_mean['YEAR']=str(data.loc[0]['YEAR'])  #stores the YEAR
   daily_mean = daily_mean.reset_index()
   daily_mean['YEAR_MONTH'] = daily_mean['YEAR']+ "_" + daily_mean['MONTH']#creates YEAR_MONTH column


   monthly_mean= daily_mean.groupby('YEAR_MONTH').mean().reset_index()     #dataframe with monthly avverages
   summary_data=pd.concat([summary_data, monthly_mean], ignore_index=True) #pd.append() has been superseded by pd.concat()


In [ ]:
#fp = r'/content/drive/Shareddrives/TRP479_Spatial_Data_Science_Data/L7/sy_summary_data.csv'
summary_data.head(10)
summary_data.tail(10)
#summary_data.to_csv(fp)